# Lab 13 — Recurrent Neural Networks

> **강의 시간:** 약 2시간  
> **주제:** 순환 신경망 — Vanilla RNN, LSTM, GRU, 시계열 예측

---

## 학습 목표

| # | 목표 | 예상 시간 |
|---|---|---|
| 1 | Vanilla RNN: 수동 구현, 학습, 그래디언트 소실 | 30분 |
| 2 | LSTM: 게이트 메커니즘, 학습, 미래 예측 | 35분 |
| 3 | GRU: 단순화된 게이팅, 3모델 비교 | 25분 |
| 4 | Exercise | 25분 |

---

**데이터셋:** numpy 합성 시계열 (인터넷 불필요)  
- 사인파: `y = sin(t) + 0.1×noise`, 1200 타임스텝  
- 복잡 신호: `sin(t) + 0.5·sin(3t) + 0.3·sin(7t) + noise`, 1600 타임스텝

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

_fp = '/System/Library/Fonts/AppleGothic.ttf'
fm.fontManager.addfont(_fp)
plt.rcParams['font.family'] = fm.FontProperties(fname=_fp).get_name()
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

torch.manual_seed(42)
np.random.seed(42)

# 디바이스 자동 선택
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f'PyTorch version: {torch.__version__}')
print(f'Using device   : {device}')

In [ ]:
# ── 데이터 생성 ──────────────────────────────────────────────────────────────
np.random.seed(42)

# 사인파 (Part 1–2)
T_sine = 1200
t_sine = np.linspace(0, 12 * np.pi, T_sine)
sine_signal = (np.sin(t_sine) + 0.1 * np.random.randn(T_sine)).astype(np.float32)

# 복잡 신호 (Part 3)
T_complex = 1600
t_complex = np.linspace(0, 16 * np.pi, T_complex)
complex_signal = (np.sin(t_complex)
                  + 0.5 * np.sin(3 * t_complex)
                  + 0.3 * np.sin(7 * t_complex)
                  + 0.1 * np.random.randn(T_complex)).astype(np.float32)

# 시퀀스 생성 함수
SEQ_LEN = 30

def make_sequences(series, seq_len=SEQ_LEN):
    """
    슬라이딩 윈도우로 (입력, 타깃) 쌍 생성.
    입력: series[i : i+seq_len]
    타깃: series[i+1 : i+seq_len+1]  (한 스텝 앞 예측)
    Returns: X (N, seq_len, 1), y (N, seq_len, 1)
    """
    X = np.array([series[i:i+seq_len]       for i in range(len(series)-seq_len)])
    y = np.array([series[i+1:i+seq_len+1]   for i in range(len(series)-seq_len)])
    return (torch.FloatTensor(X).unsqueeze(-1),
            torch.FloatTensor(y).unsqueeze(-1))

def split_data(X, y, train_ratio=0.8):
    n = int(len(X) * train_ratio)
    return X[:n], y[:n], X[n:], y[n:]

# 사인파 데이터 준비
X_sine, y_sine = make_sequences(sine_signal)
X_tr, y_tr, X_te, y_te = split_data(X_sine, y_sine)

train_ds  = TensorDataset(X_tr, y_tr)
train_ldr = DataLoader(train_ds, batch_size=64, shuffle=True)

print('=== 사인파 데이터셋 ===')
print(f'  총 타임스텝 : {T_sine}')
print(f'  seq_len     : {SEQ_LEN}')
print(f'  총 시퀀스   : {len(X_sine)}  (X shape: {tuple(X_sine.shape)})')
print(f'  Train       : {len(X_tr)}  / Test: {len(X_te)}')

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 3.5))
axes[0].plot(t_sine[:200], sine_signal[:200], color='steelblue', lw=1.5)
axes[0].set_title('사인파 신호 (처음 200 타임스텝)'); axes[0].set_xlabel('t'); axes[0].set_ylabel('y')

axes[1].plot(t_complex[:400], complex_signal[:400], color='tomato', lw=1.5)
axes[1].set_title('복잡 신호 (처음 400 타임스텝)'); axes[1].set_xlabel('t'); axes[1].set_ylabel('y')

plt.suptitle('학습 데이터 시각화', fontsize=12)
plt.tight_layout()
plt.show()

---
## Part 1. Vanilla RNN

### 1-1. RNN의 핵심 수식

RNN은 이전 타임스텝의 은닉 상태를 현재 입력과 결합합니다:

$$h_t = \tanh(W_{ih} \cdot x_t + W_{hh} \cdot h_{t-1} + b)$$
$$\hat{y}_t = W_{ho} \cdot h_t + b_o$$

| 기호 | 크기 | 설명 |
|---|---|---|
| $x_t$ | (input_size,) | 타임스텝 $t$의 입력 |
| $h_t$ | (hidden_size,) | 타임스텝 $t$의 은닉 상태 |
| $W_{ih}$ | (hidden, input) | 입력→은닉 가중치 |
| $W_{hh}$ | (hidden, hidden) | 은닉→은닉 순환 가중치 (핵심!) |

### 1-2. 핵심 특징
- **가중치 공유**: 모든 타임스텝에서 동일한 $W_{ih}, W_{hh}, b$ 사용
- **시간 전개(Unrolling)**: 시퀀스를 따라 펼치면 매우 깊은 네트워크와 동일
- **BPTT**: 시간 역전파(Backpropagation Through Time)로 학습

In [ ]:
# RNN 수동 구현 (numpy) — nn.RNN과 동일한 연산

def rnn_step_manual(x_t, h_prev, W_ih, W_hh, b):
    """단일 타임스텝 RNN 업데이트
    h_t = tanh(W_ih @ x_t + W_hh @ h_prev + b)
    """
    return np.tanh(W_ih @ x_t + W_hh @ h_prev + b)


# 파라미터 설정 (input=1, hidden=4)
np.random.seed(0)
input_size, hidden_size = 1, 4
W_ih = np.random.randn(hidden_size, input_size)  * 0.3
W_hh = np.random.randn(hidden_size, hidden_size) * 0.3
b    = np.zeros(hidden_size)

# 사인파 20 타임스텝으로 순전파
t_demo    = np.linspace(0, 2 * np.pi, 20)
x_demo    = np.sin(t_demo)
h         = np.zeros(hidden_size)
hiddens   = []

for val in x_demo:
    h = rnn_step_manual(np.array([val]), h, W_ih, W_hh, b)
    hiddens.append(h.copy())

hiddens = np.array(hiddens)   # (20, 4)

# PyTorch nn.RNN으로 동일 파라미터 검증
rnn_check = nn.RNN(input_size, hidden_size, batch_first=True)
with torch.no_grad():
    rnn_check.weight_ih_l0.copy_(torch.FloatTensor(W_ih))
    rnn_check.weight_hh_l0.copy_(torch.FloatTensor(W_hh))
    rnn_check.bias_ih_l0.zero_()
    rnn_check.bias_hh_l0.zero_()
    x_pt  = torch.FloatTensor(x_demo).view(1, -1, 1)
    out_pt, _ = rnn_check(x_pt)

max_diff = np.max(np.abs(hiddens - out_pt[0].numpy()))
print(f'수동 RNN vs nn.RNN 최대 오차: {max_diff:.2e}  (≈0 이면 동일)')

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(13, 4))
axes[0].plot(t_demo, x_demo, 'b-o', ms=4, lw=2, label='입력 x_t')
axes[0].set_title('입력 시퀀스 (사인파, 20 타임스텝)')
axes[0].set_xlabel('t'); axes[0].set_ylabel('값'); axes[0].legend()

colors_h = ['steelblue', 'tomato', 'seagreen', 'darkorange']
for i in range(hidden_size):
    axes[1].plot(hiddens[:, i], color=colors_h[i], lw=2, label=f'h[{i}]')
axes[1].set_title('은닉 상태 h_t (hidden_size=4)')
axes[1].set_xlabel('타임스텝'); axes[1].set_ylabel('활성화값'); axes[1].legend()

plt.suptitle('Vanilla RNN 수동 순전파: h_t = tanh(W_ih·x_t + W_hh·h_{t-1} + b)', fontsize=11)
plt.tight_layout()
plt.show()

print(f'\n파라미터 구조:')
print(f'  W_ih : {W_ih.shape}  (hidden_size × input_size)')
print(f'  W_hh : {W_hh.shape}  (hidden_size × hidden_size) ← 순환 연결')
print(f'  b    : {b.shape}')
print(f'  총 파라미터: {W_ih.size + W_hh.size + b.size}개')

In [ ]:
# VanillaRNN 클래스 및 학습 함수 정의

class VanillaRNN(nn.Module):
    """Many-to-Many Vanilla RNN: 각 타임스텝의 다음 값 예측"""
    def __init__(self, input_size=1, hidden_size=32, output_size=1, n_layers=1):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, n_layers,
                          batch_first=True, nonlinearity='tanh')
        self.fc  = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)   # out: (B, T, hidden)
        return self.fc(out)    # (B, T, 1)


class LSTMNet(nn.Module):
    """Many-to-Many LSTM"""
    def __init__(self, input_size=1, hidden_size=32, output_size=1, n_layers=1, dropout=0.0):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, n_layers,
                            batch_first=True,
                            dropout=dropout if n_layers > 1 else 0.0)
        self.fc   = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)  # out: (B, T, hidden)
        return self.fc(out)    # (B, T, 1)


class GRUNet(nn.Module):
    """Many-to-Many GRU"""
    def __init__(self, input_size=1, hidden_size=32, output_size=1, n_layers=1, dropout=0.0):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, n_layers,
                          batch_first=True,
                          dropout=dropout if n_layers > 1 else 0.0)
        self.fc  = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)   # out: (B, T, hidden)
        return self.fc(out)    # (B, T, 1)


def count_params(model):
    return sum(p.numel() for p in model.parameters())


def train_rnn(model, train_loader, X_val, y_val,
              n_epochs=50, lr=0.001, verbose=True):
    """범용 RNN 학습 함수 (MSE 손실, Gradient Clipping)"""
    model     = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    X_v = X_val.to(device)
    y_v = y_val.to(device)

    train_losses, val_losses = [], []

    for epoch in range(n_epochs):
        model.train()
        ep_loss = 0.
        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            optimizer.zero_grad()
            pred = model(Xb)
            loss = criterion(pred, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # 그래디언트 클리핑
            optimizer.step()
            ep_loss += loss.item()

        train_losses.append(ep_loss / len(train_loader))

        model.eval()
        with torch.no_grad():
            val_pred = model(X_v)
            val_losses.append(criterion(val_pred, y_v).item())

        if verbose and (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1:3d}/{n_epochs} | '
                  f'Train MSE: {train_losses[-1]:.6f} | '
                  f'Val MSE: {val_losses[-1]:.6f}')

    model.cpu()
    return train_losses, val_losses


def predict_future(model, seed_seq, n_steps=100):
    """자기회귀 미래 예측: seed_seq (seq_len, 1) 텐서로 n_steps 예측"""
    model.eval()
    preds   = []
    current = seed_seq.unsqueeze(0).to(device)  # (1, seq_len, 1)
    with torch.no_grad():
        for _ in range(n_steps):
            out  = model(current)          # (1, seq_len, 1)
            next_val = out[:, -1:, :]      # (1, 1, 1) — 마지막 타임스텝만
            preds.append(next_val.squeeze().item())
            current = torch.cat([current[:, 1:, :], next_val], dim=1)  # 시퀀스 한 칸 이동
    model.cpu()
    return preds


# 파라미터 수 비교
for cls, name in [(VanillaRNN, 'VanillaRNN'), (LSTMNet, 'LSTMNet'), (GRUNet, 'GRUNet')]:
    m = cls(hidden_size=32)
    print(f'{name:<12}: {count_params(m):,} 파라미터')

In [ ]:
# Vanilla RNN 학습 — 사인파 다음 스텝 예측
print('=== Vanilla RNN 학습 (50 에포크) ===')
torch.manual_seed(42)
model_rnn = VanillaRNN(hidden_size=32)
rnn_tr_loss, rnn_val_loss = train_rnn(
    model_rnn, train_ldr, X_te, y_te, n_epochs=50
)

# 예측 시각화
model_rnn.eval()
with torch.no_grad():
    pred_rnn = model_rnn(X_te).squeeze(-1).numpy()  # (N, seq_len)

# 테스트셋 처음 200 타임스텝 시각화
true_vals = y_te[:, :, 0].numpy().flatten()[:200]
pred_vals = pred_rnn.flatten()[:200]

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(range(1, 51), rnn_tr_loss,  'b-', lw=2, label='Train MSE')
axes[0].plot(range(1, 51), rnn_val_loss, 'r-', lw=2, label='Val MSE')
axes[0].set_title('학습 곡선'); axes[0].set_xlabel('에포크')
axes[0].set_ylabel('MSE'); axes[0].legend()

axes[1].plot(true_vals, 'b-', lw=1.5, alpha=0.8, label='실제')
axes[1].plot(pred_vals, 'r--', lw=1.5, alpha=0.8, label='예측 (RNN)')
axes[1].set_title('테스트셋 예측 (처음 200 타임스텝)')
axes[1].set_xlabel('타임스텝'); axes[1].set_ylabel('값'); axes[1].legend()

plt.suptitle(f'Vanilla RNN 사인파 예측 (Val MSE: {rnn_val_loss[-1]:.6f})', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# 그래디언트 소실 (Vanishing Gradient) 시각화
# 랜덤 초기화된 RNN/LSTM에서 마지막 출력의 각 입력 타임스텝별 그래디언트 측정

SEQ_GRAD = 60  # 긴 시퀀스
torch.manual_seed(0)

rnn_vg  = nn.RNN(1, 32, batch_first=True)
lstm_vg = nn.LSTM(1, 32, batch_first=True)
gru_vg  = nn.GRU(1, 32, batch_first=True)

results_vg = {}

for model_layer, name in [(rnn_vg, 'Vanilla RNN'), (lstm_vg, 'LSTM'), (gru_vg, 'GRU')]:
    x = torch.randn(1, SEQ_GRAD, 1, requires_grad=True)
    out, _ = model_layer(x)
    # 마지막 타임스텝 출력의 합을 스칼라 손실로 사용
    out[:, -1, :].sum().backward()
    # 각 입력 타임스텝에서의 그래디언트 크기
    grad_norms = x.grad[0, :, 0].abs().detach().numpy()
    results_vg[name] = grad_norms

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
colors_vg = {'Vanilla RNN': 'tomato', 'LSTM': 'seagreen', 'GRU': 'steelblue'}

# 선형 스케일
for name, grads in results_vg.items():
    axes[0].plot(grads, color=colors_vg[name], lw=2, label=name)
axes[0].set_title('그래디언트 크기 (선형 스케일)')
axes[0].set_xlabel('입력 타임스텝 t'); axes[0].set_ylabel('|∂L/∂x_t|')
axes[0].legend()

# 로그 스케일 (소실 효과 더 잘 보임)
for name, grads in results_vg.items():
    axes[1].semilogy(grads + 1e-20, color=colors_vg[name], lw=2, label=name)
axes[1].set_title('그래디언트 크기 (로그 스케일)')
axes[1].set_xlabel('입력 타임스텝 t'); axes[1].set_ylabel('|∂L/∂x_t| (log)')
axes[1].legend()

plt.suptitle(f'그래디언트 소실 비교 (seq_len={SEQ_GRAD})\n'
             '초기 타임스텝(왼쪽)일수록 그래디언트가 작다 = 학습이 어렵다', fontsize=11)
plt.tight_layout()
plt.show()

print('=== 초기 10 타임스텝 평균 그래디언트 크기 ===')
for name, grads in results_vg.items():
    print(f'  {name:<12}: {grads[:10].mean():.2e}')

print()
print('📌 핵심:')
print('  - Vanilla RNN: 초기 타임스텝 그래디언트 ≈ 0 → 장기 의존성 학습 불가')
print('  - LSTM/GRU   : 셀 상태(또는 게이트)가 그래디언트 "통로" 역할 → 장기 의존성 유지')

---
## Part 2. LSTM (Long Short-Term Memory)

### 2-1. LSTM 게이트 메커니즘

LSTM은 **셀 상태(Cell State) $C_t$** 라는 별도의 정보 흐름을 유지합니다.
3가지 게이트가 정보의 흐름을 제어합니다:

$$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f) \quad \text{(망각 게이트: 무엇을 잊을지)}$$
$$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i) \quad \text{(입력 게이트: 무엇을 저장할지)}$$
$$g_t = \tanh(W_g \cdot [h_{t-1}, x_t] + b_g) \quad \text{(후보 값)}$$
$$o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o) \quad \text{(출력 게이트: 무엇을 출력할지)}$$

$$C_t = f_t \odot C_{t-1} + i_t \odot g_t \quad \text{(셀 상태 업데이트)}$$
$$h_t = o_t \odot \tanh(C_t) \quad \text{(은닉 상태)}$$

**핵심**: $C_t$는 **덧셈 연산**으로 업데이트 → 그래디언트가 사라지지 않고 흐를 수 있음!

| 항목 | Vanilla RNN | LSTM |
|---|---|---|
| 상태 | $h_t$ 1개 | $h_t$ + $C_t$ 2개 |
| 게이트 | 없음 | 3개 (forget, input, output) |
| 파라미터 | $4 \times \text{hidden}^2$ | $16 \times \text{hidden}^2$ (4배) |
| 장기 의존성 | 취약 | 강함 |

In [ ]:
# LSTM 학습 — 사인파 예측
print('=== LSTM 학습 (50 에포크) ===')
torch.manual_seed(42)
model_lstm = LSTMNet(hidden_size=32)
lstm_tr_loss, lstm_val_loss = train_rnn(
    model_lstm, train_ldr, X_te, y_te, n_epochs=50
)

# 비교 시각화
model_lstm.eval()
with torch.no_grad():
    pred_lstm = model_lstm(X_te).squeeze(-1).numpy()

ep = range(1, 51)
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# 학습 곡선
axes[0].plot(ep, rnn_tr_loss,   'tomato',   lw=2, ls='--', label='RNN Train')
axes[0].plot(ep, rnn_val_loss,  'tomato',   lw=2, label='RNN Val')
axes[0].plot(ep, lstm_tr_loss,  'seagreen', lw=2, ls='--', label='LSTM Train')
axes[0].plot(ep, lstm_val_loss, 'seagreen', lw=2, label='LSTM Val')
axes[0].set_title('학습 곡선 (MSE)'); axes[0].set_xlabel('에포크')
axes[0].set_ylabel('MSE'); axes[0].legend(fontsize=8)

# RNN 예측
n_show = 150
true_flat = y_te[:, :, 0].numpy().flatten()[:n_show]
axes[1].plot(true_flat,                   'b-',  lw=1.5, label='실제')
axes[1].plot(pred_rnn.flatten()[:n_show], 'r--', lw=1.5, label=f'RNN ({rnn_val_loss[-1]:.5f})')
axes[1].set_title('RNN 예측'); axes[1].set_xlabel('타임스텝'); axes[1].legend(fontsize=9)

# LSTM 예측
axes[2].plot(true_flat,                    'b-',  lw=1.5, label='실제')
axes[2].plot(pred_lstm.flatten()[:n_show], 'g--', lw=1.5, label=f'LSTM ({lstm_val_loss[-1]:.5f})')
axes[2].set_title('LSTM 예측'); axes[2].set_xlabel('타임스텝'); axes[2].legend(fontsize=9)

plt.suptitle('Vanilla RNN vs LSTM — 사인파 예측 비교', fontsize=12)
plt.tight_layout()
plt.show()

print(f'\n최종 Val MSE:')
print(f'  RNN  : {rnn_val_loss[-1]:.6f}')
print(f'  LSTM : {lstm_val_loss[-1]:.6f}')

In [ ]:
# 자기회귀 미래 예측 (Autoregressive Multi-step Prediction)
# 마지막 테스트 시퀀스를 시드로 100 스텝 앞을 예측

seed_seq   = X_te[-1]               # (seq_len, 1) — 마지막 테스트 시퀀스
n_future   = 100

future_rnn  = predict_future(model_rnn,  seed_seq, n_steps=n_future)
future_lstm = predict_future(model_lstm, seed_seq, n_steps=n_future)

# 비교 시각화
# 시드 뒤의 실제 사인파값
seed_start_idx = len(sine_signal) - len(X_te) + (len(X_te) - 1) + SEQ_LEN
true_future    = sine_signal[seed_start_idx:seed_start_idx + n_future] \
                 if seed_start_idx + n_future <= len(sine_signal) \
                 else np.full(n_future, np.nan)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
seed_vals = seed_seq[:, 0].numpy()

for ax, (name, future), color in [
    (axes[0], ('RNN',  future_rnn),  'tomato'),
    (axes[1], ('LSTM', future_lstm), 'seagreen'),
]:
    # 시드 시퀀스
    ax.plot(range(SEQ_LEN), seed_vals, 'b-', lw=2, label='시드 (실제)')
    # 미래 예측
    ax.plot(range(SEQ_LEN, SEQ_LEN + n_future), future, color=color, lw=2,
            ls='--', label=f'{name} 예측')
    # 실제 미래값 (가능한 경우)
    if not np.isnan(true_future).all():
        ax.plot(range(SEQ_LEN, SEQ_LEN + n_future), true_future,
                'gray', lw=1.5, alpha=0.7, label='실제 미래')
    ax.axvline(x=SEQ_LEN, color='black', ls=':', lw=1.5, label='예측 시작')
    ax.set_title(f'{name}: {n_future}스텝 미래 예측')
    ax.set_xlabel('타임스텝'); ax.set_ylabel('값'); ax.legend(fontsize=8)

plt.suptitle('자기회귀 미래 예측: RNN vs LSTM', fontsize=12)
plt.tight_layout()
plt.show()

print('📌 LSTM은 장기 패턴(사인파 주기)을 더 잘 유지합니다.')

---
## Part 3. GRU (Gated Recurrent Unit)

### 3-1. GRU: LSTM의 경량화 버전

2014년 Cho et al.이 제안한 GRU는 LSTM을 **단순화**하여 파라미터를 줄입니다.

$$r_t = \sigma(W_r \cdot [h_{t-1}, x_t]) \quad \text{(리셋 게이트: 과거를 얼마나 리셋)}$$
$$z_t = \sigma(W_z \cdot [h_{t-1}, x_t]) \quad \text{(업데이트 게이트: 얼마나 업데이트)}$$
$$\tilde{h}_t = \tanh(W \cdot [r_t \odot h_{t-1},\; x_t]) \quad \text{(후보 은닉 상태)}$$
$$h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t$$

| 비교 | LSTM | GRU |
|---|---|---|
| 상태 | $h_t$ + $C_t$ | $h_t$ 만 |
| 게이트 수 | 3개 (forget, input, output) | 2개 (reset, update) |
| 파라미터 | $4 \times 4 \times H^2$ | $3 \times 4 \times H^2$ (25% 절감) |
| 성능 | 복잡한 패턴에 유리 | 작은 데이터, 빠른 학습에 유리 |

In [ ]:
# GRU 학습 및 3모델 비교 (사인파)
print('=== GRU 학습 (50 에포크) ===')
torch.manual_seed(42)
model_gru = GRUNet(hidden_size=32)
gru_tr_loss, gru_val_loss = train_rnn(
    model_gru, train_ldr, X_te, y_te, n_epochs=50
)

# 3모델 비교 시각화
model_gru.eval()
with torch.no_grad():
    pred_gru = model_gru(X_te).squeeze(-1).numpy()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
ep = range(1, 51)

for (name, tl, vl, color) in [
    ('RNN',  rnn_tr_loss,  rnn_val_loss,  'tomato'),
    ('LSTM', lstm_tr_loss, lstm_val_loss, 'seagreen'),
    ('GRU',  gru_tr_loss,  gru_val_loss,  'steelblue'),
]:
    axes[0].plot(ep, tl, color=color, lw=1.5, ls='--', alpha=0.6)
    axes[0].plot(ep, vl, color=color, lw=2, label=f'{name} Val ({vl[-1]:.5f})')

axes[0].set_title('3모델 학습 곡선 (Val MSE, —)')
axes[0].set_xlabel('에포크'); axes[0].set_ylabel('MSE')
axes[0].legend(fontsize=9)

n_show = 150
true_flat = y_te[:, :, 0].numpy().flatten()[:n_show]
axes[1].plot(true_flat,                    'b-',  lw=2,   label='실제')
axes[1].plot(pred_rnn.flatten()[:n_show],  'r--', lw=1.5, label='RNN')
axes[1].plot(pred_lstm.flatten()[:n_show], 'g:',  lw=2,   label='LSTM')
axes[1].plot(pred_gru.flatten()[:n_show],  color='steelblue', ls='-.', lw=1.5, label='GRU')
axes[1].set_title('테스트셋 예측 비교'); axes[1].set_xlabel('타임스텝')
axes[1].set_ylabel('값'); axes[1].legend(fontsize=9)

plt.suptitle('Vanilla RNN vs LSTM vs GRU — 사인파 예측', fontsize=12)
plt.tight_layout()
plt.show()

# 성능 요약 표
print(f'\n{"모델":<10} {"Val MSE":>12} {"파라미터":>10}')
print('-' * 36)
for name, vl, m in [
    ('RNN',  rnn_val_loss,  model_rnn),
    ('LSTM', lstm_val_loss, model_lstm),
    ('GRU',  gru_val_loss,  model_gru),
]:
    mark = ' ★' if vl[-1] == min(rnn_val_loss[-1], lstm_val_loss[-1], gru_val_loss[-1]) else ''
    print(f'{name:<10} {vl[-1]:>12.6f} {count_params(m):>10,}{mark}')

In [ ]:
# 복잡 신호(다중 주파수)에서 3모델 비교
# sin(t) + 0.5·sin(3t) + 0.3·sin(7t) + noise

X_cx, y_cx = make_sequences(complex_signal)
X_cx_tr, y_cx_tr, X_cx_te, y_cx_te = split_data(X_cx, y_cx)

cx_ds  = TensorDataset(X_cx_tr, y_cx_tr)
cx_ldr = DataLoader(cx_ds, batch_size=64, shuffle=True)

cx_results = {}
print('=== 복잡 신호 학습 (50 에포크) ===\n')
for cls, name in [(VanillaRNN, 'RNN'), (LSTMNet, 'LSTM'), (GRUNet, 'GRU')]:
    torch.manual_seed(42)
    m = cls(hidden_size=64)
    print(f'--- {name} ---')
    tl, vl = train_rnn(m, cx_ldr, X_cx_te, y_cx_te, n_epochs=50)
    m.eval()
    with torch.no_grad():
        pred = m(X_cx_te).squeeze(-1).numpy()
    cx_results[name] = (tl, vl, pred, m)
    print()

# 시각화
n_show = 200
true_cx = y_cx_te[:, :, 0].numpy().flatten()[:n_show]
ep = range(1, 51)
colors_cx = {'RNN': 'tomato', 'LSTM': 'seagreen', 'GRU': 'steelblue'}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for name, (tl, vl, pred, m) in cx_results.items():
    axes[0].plot(ep, vl, color=colors_cx[name], lw=2, label=f'{name} ({vl[-1]:.5f})')

axes[0].set_title('복잡 신호 Val MSE')
axes[0].set_xlabel('에포크'); axes[0].set_ylabel('MSE')
axes[0].legend(fontsize=9)

axes[1].plot(true_cx, 'b-', lw=2, alpha=0.9, label='실제')
for name, (_, vl, pred, _) in cx_results.items():
    axes[1].plot(pred.flatten()[:n_show], color=colors_cx[name], lw=1.5,
                 ls='--', label=f'{name} ({vl[-1]:.5f})')
axes[1].set_title('복잡 신호 테스트 예측')
axes[1].set_xlabel('타임스텝'); axes[1].set_ylabel('값')
axes[1].legend(fontsize=9)

plt.suptitle('복잡 신호: sin(t)+0.5·sin(3t)+0.3·sin(7t)+noise', fontsize=12)
plt.tight_layout()
plt.show()

print(f'\n{"모델":<6} {"Val MSE (복잡 신호)":>22} {"파라미터":>10}')
print('-' * 44)
for name, (_, vl, _, m) in cx_results.items():
    print(f'{name:<6} {vl[-1]:>22.6f} {count_params(m):>10,}')

---
## Exercise

### Exercise 1. LSTM 하이퍼파라미터 탐색

`hidden_size`와 `n_layers`를 조합하여 최적 LSTM 설정을 찾으세요.

| hidden_size | n_layers | 비고 |
|---|---|---|
| 16 | 1 | |
| 32 | 1 | (Part 2와 동일) |
| 64 | 1 | |
| 16 | 2 | |
| 32 | 2 | |
| 64 | 2 | |

**요구사항:**
- 데이터: 사인파 (`train_ldr`, `X_te`, `y_te`)
- 동일 조건: n_epochs=50, Adam, lr=0.001
- 최종 Val MSE + 파라미터 수를 표로 출력
- 결과를 막대그래프 또는 히트맵으로 시각화

In [ ]:
# Exercise 1: LSTM 하이퍼파라미터 탐색

hp_configs = [
    {'hidden_size': 16, 'n_layers': 1},
    {'hidden_size': 32, 'n_layers': 1},
    {'hidden_size': 64, 'n_layers': 1},
    {'hidden_size': 16, 'n_layers': 2},
    {'hidden_size': 32, 'n_layers': 2},
    {'hidden_size': 64, 'n_layers': 2},
]

hp_results = {}

for cfg in hp_configs:
    key = f'h={cfg["hidden_size"]}, L={cfg["n_layers"]}'
    # Your code here: LSTMNet 생성 및 학습
    # torch.manual_seed(42)
    # m = LSTMNet(**cfg)
    # tl, vl = train_rnn(m, train_ldr, X_te, y_te, n_epochs=50, verbose=False)
    # hp_results[key] = (vl[-1], count_params(m))
    pass

# Your code here: 결과 표 출력
# print(f'{"설정":<20} {"Val MSE":>12} {"파라미터":>10}')
# ...

# Your code here: 막대그래프 시각화 (x=설정, y=Val MSE)

### Exercise 2. 복잡 신호 자기회귀 예측

복잡 신호(`complex_signal`)에서 최적 LSTM을 학습하고 **100스텝 미래를 자기회귀 방식**으로 예측하세요.

**요구사항:**
1. `cx_ldr`, `X_cx_te`, `y_cx_te`로 LSTM 학습 (n_epochs=50, hidden_size=64)
2. `predict_future()` 함수로 100스텝 미래 예측
3. 시각화: Train/Val Loss 곡선 + 시드 시퀀스 + 예측 미래값
4. 사인파 예측 때와 비교: 더 어렵습니까, 쉽습니까?

In [ ]:
# Exercise 2: 복잡 신호 자기회귀 예측

# Your code here: LSTM 학습 (cx_ldr, X_cx_te, y_cx_te, hidden_size=64)
torch.manual_seed(42)
model_cx_lstm = LSTMNet(hidden_size=64)

# cx_tl, cx_vl = train_rnn(model_cx_lstm, cx_ldr, X_cx_te, y_cx_te, n_epochs=50)

# Your code here: 마지막 테스트 시퀀스로 100스텝 미래 예측
# seed = X_cx_te[-1]  # (seq_len, 1)
# future_cx = predict_future(model_cx_lstm, seed, n_steps=100)

# Your code here: 시각화
# - 왼쪽: Train/Val Loss 곡선
# - 오른쪽: 시드(파란색) + 예측 미래(빨간 점선)

### Exercise 3. (도전) 시퀀스 분류 — LSTM Classifier

**추세 상승(↑) vs 추세 하강(↓)** 이진 분류 과제를 LSTM으로 풀어보세요.

**데이터**: 합성 시퀀스 (seq_len=30, noise 포함)
```
클래스 1 (상승): 누적합 + noise → 전체적으로 증가하는 패턴
클래스 0 (하강): -(누적합) + noise → 전체적으로 감소하는 패턴
```

**요구사항:**
1. `generate_trend_sequences()` 함수 구현 (1000 샘플 생성)
2. `LSTMClassifier` 클래스 구현: 마지막 타임스텝의 은닉 상태로 분류 (many-to-one)
3. `CrossEntropyLoss`로 학습 (n_epochs=30)
4. 테스트 정확도 출력 및 혼동 행렬 시각화

**힌트:**
```python
# many-to-one: 마지막 타임스텝의 출력만 사용
out, (h, c) = self.lstm(x)
return self.fc(out[:, -1, :])   # (B, n_classes)
```

In [ ]:
# Exercise 3: 시퀀스 분류

def generate_trend_sequences(n_samples=1000, seq_len=30, noise=0.3):
    """
    추세 상승(label=1) / 추세 하강(label=0) 시퀀스 생성
    Returns: X (N, seq_len, 1), y (N,) LongTensor
    """
    # Your code here:
    # - trend = +1 또는 -1 (0.5 확률)
    # - signal = trend * cumsum(abs(randn(seq_len))) / seq_len + noise * randn(seq_len)
    # - 각 시퀀스를 표준화 (mean=0, std=1)
    # - label: trend > 0 이면 1, 아니면 0
    pass


class LSTMClassifier(nn.Module):
    """Many-to-One LSTM 분류기: 마지막 은닉 상태로 클래스 예측"""
    def __init__(self, input_size=1, hidden_size=32, n_layers=1, n_classes=2):
        super().__init__()
        # Your code here: nn.LSTM + nn.Linear
        pass

    def forward(self, x):
        # Your code here: LSTM → 마지막 타임스텝 출력 → Linear
        pass


# Your code here:
# 1. generate_trend_sequences()로 데이터 생성 (train 800, test 200)
# 2. LSTMClassifier 생성 및 CrossEntropyLoss로 학습 (n_epochs=30)
# 3. 테스트 정확도 출력
# 4. 혼동 행렬(confusion matrix) 시각화

---
## Summary

| 개념 | 핵심 내용 |
|---|---|
| **Vanilla RNN** | $h_t = \tanh(W_{ih}x_t + W_{hh}h_{t-1} + b)$; 가중치 공유, BPTT |
| **그래디언트 소실** | $W_{hh}$를 반복 곱 → 초기 타임스텝 그래디언트 ≈ 0; 장기 의존성 학습 불가 |
| **LSTM** | 셀 상태 $C_t$ + 3게이트(forget/input/output); 덧셈 업데이트로 그래디언트 보존 |
| **망각 게이트** | $f_t = \sigma(\cdot)$; 과거 정보를 얼마나 유지할지 |
| **GRU** | LSTM 단순화; 2게이트(reset/update), 셀 상태 없음, 파라미터 25% 절감 |
| **Many-to-Many** | 각 타임스텝 출력 사용 → 시퀀스 예측 (시계열, 번역) |
| **Many-to-One** | 마지막 출력만 사용 → 시퀀스 분류 (감성 분석, 추세 분류) |
| **자기회귀 예측** | 예측값을 다음 입력으로 재사용 → 장기 미래 예측 (오차 누적) |
| **Gradient Clipping** | `clip_grad_norm_(..., 1.0)`; RNN 그래디언트 폭발 방지 |
| **Seq2Seq** | Encoder(RNN) → context vector → Decoder(RNN); 번역/요약 기반 아이디어 |

---

**다음 강의 (Week 14):** Transformer — Attention 메커니즘, Self-Attention, BERT/GPT 개요